# Identify H-bearing reactions that should have deuterated analogues

This notebook is used to find which reaction rates are most important for producing H-bearing species, for the purpose of producing a deuterated network.

In [1]:
using Revise
photochemistry_source_dir = "$(@__DIR__)/Photochemistry/src/"
push!(LOAD_PATH, photochemistry_source_dir)
using Photochemistry  # custom module
using DataFrames

In [2]:
include("CONSTANTS.jl");
include("CUSTOMIZATIONS.jl");

In [3]:
basic_case_folder = "/home/emc/OneDrive-CU/Research/UpperAtmoDH/Results/DGearI+iNT_bXwiopqk_smean_v13/"
ncur = get_ncurrent(basic_case_folder*"final_atmosphere.h5");

vardict = load_from_paramlog(basic_case_folder);

reaction_network, hhn, hdn, hh2n, hhdn = load_reaction_network(vardict["rxn_spreadsheet"]; get_hot_rxns=true, ions_on=true, all_species=vardict["all_species"]);

# Function to print dominant reactions

In [4]:
function get_column_rates_NEW(sp::Symbol, atmdict::Dict{Symbol, Vector{ftype_ncur}}; which="all", role="both", startalt_i=1, globvars...)
    #=
    Input:
        sp: species for which to search for reactions
        atmdict: the present atmospheric state to calculate on
        Tn, Ti, Te: Arrays of the temperature profiles including boundary layers
        bcdict: Boundary conditions dictionary specified in parameters file
        which: whether to do photochemistry, or just bimolecular reactions. "all", "Jrates" or "krates"
        sp2: optional second species to include, i.e. usually sp's ion.
        role: "product" or "reactant" only.
        startalt_i: Index of the altitude at which to start. This lets you only calculate column rate down to a certain altitude, which is
                    useful, for example, for water, which is fixed at 80 km so we don't care what produces/consumes it.
    Output:
        sorted: Total column rates for all reactions of species sp. Sorted, in order of largest rate to smallest. NOT a dictionary.
                sorted[1] is the top production mechanism, e.g.
    =#
    GV = values(globvars)
    @assert all(x->x in keys(GV), [:Tn, :Ti, :Te, :all_species, :ion_species, :reaction_network, :num_layers, :dz])
    
    # println("Column rates will be collected for reaction type = $(which) and species role = $(role)")
    rxd, coefs = get_volume_rates(sp, atmdict, n_horiz; species_role=role, which=which, globvars...)

    
    # rxd comes out as a dictionary; convert it to a dataframe
    df = DataFrame(rxd)
    
    # Now sum each reaction over altitudes and multiply by dz to get the column rate
    dfsum = dfsum = mapcols(sum, df) .* GV.dz
    # Now transpose and sort 
    df_transposed = DataFrame([[names(dfsum)]; collect.(eachrow(dfsum))], [:Rxn; :Value])
    df_sorted = sort(df_transposed, [:Value], rev=true)
    
    return df_sorted
end

# thecoldf = get_column_rates_NEW(:H, ncur; which="all", sp2=nothing, Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
#                                    all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
#                                    num_layers, dz)

get_column_rates_NEW (generic function with 1 method)

In [38]:
function get_p_pct_of_column(col_rate_df, p; sp=nothing, globvars...)
    #=
    This will calculate the column rate for the top p% of reactions, and
    return the subset of col_rate_df for that number of reactions, starting from the highest column rate.
    
    col_rate_df: a dataframe containing various chemical reactions and their column rate.
    p: we wish to get the top x reactions that contribute to p% of the total column rate.
    =#
    
    GV = values(globvars)
    @assert all(x->x in keys(GV), [:Tn, :Ti, :Te, :all_species, :ion_species, :reaction_network, :num_layers, :dz])
    
    if p > 1
        throw("Please enter percent as a decimal, i.e. 1 = 100%, 0.01 = 1%.")
    end
    
    total_col_rate_all_rxns = sum(col_rate_df."Value")
    
    # Figure out what percent of total column rate the top x% reactions are.
    if nrow(col_rate_df) <= 25
        if sp != nothing
            x = nrow(col_rate_df) # set total num of reactions to all of them
            println("$(sp) has a small number of reactions ($(x)). Recommended to create a deuterated analogue for all of them.")
            p = 1 # 100% of column rate is represented.
        else
            # if we didn't specify a species, we can still go ahead and calculate what x we require for p% to be represented.
            x = Int(floor(nrow(col_rate_df) * p))
            
        end
    else
        x = Int(floor(nrow(col_rate_df) * p))
    end

    # Handle case where we might round down too hard
    if x < 1
        x = 1
        println("Error: $(p*100)% of reactions is 0, resetting to print out only one reaction.")
        
        p = x / nrow(col_rate_df)
        println("Printing 1 reaction ($(round(p*100, digits=2))% of reactions).")
    else
        println("$(p*100)% of reactions is $(x)/$(nrow(col_rate_df))")
    end
    
    col_rate_of_top_p_rxns = sum(col_rate_df."Value"[1:x])
    println("The column rate of top $(round(p*100, digits=2))% of reactions is $(col_rate_of_top_p_rxns)")
    println("Which equates to $(100*col_rate_of_top_p_rxns / total_col_rate_all_rxns)% of the total column rate ($(total_col_rate_all_rxns)).")
    println()
    println(col_rate_df[1:x, :])
    
    return col_rate_df[1:x, :]
end

function rxns_comprising_p_pct_colrate(col_rate_df, p; sp=nothing, globvars...)
    #=
    This will return the subset of col_rate_df which comprises 99% of the column rate total for the whole df.
    
    col_rate_df: a dataframe containing various chemical reactions and their column rate.
    p: we wish to get the top x reactions that contribute to p% of the total column rate.
    =#
    
    GV = values(globvars)
    @assert all(x->x in keys(GV), [:Tn, :Ti, :Te, :all_species, :ion_species, :reaction_network, :num_layers, :dz])
    
    if p > 1
        throw("Please enter percent as a decimal, i.e. 1 = 100%, 0.01 = 1%.")
    end
    
    total_col_rate_all_rxns = sum(col_rate_df."Value")
    
    p_times_total_columnrate = p * total_col_rate_all_rxns

    this_pct = 1
    i = nrow(col_rate_df)+1
    while this_pct > p
        i -= 1
        this_pct = sum(col_rate_df[1:i, :]."Value") / total_col_rate_all_rxns
        println("$(i) is $(100*this_pct)%")
        
        if this_pct < 0.999
            i += 1
        elseif this_pct == 0.999
            continue
        end
    end
    
    println("$(i) reactions needed to make up $(100*p)% of col rate")

    return col_rate_df[1:i, :]
end

rxns_comprising_p_pct_colrate (generic function with 1 method)

# H-bearing species list

In [111]:
H_bearing_species = [s for s in vardict["all_species"] if occursin('H', string(s))];

# Get 100% of reactions for all H-bearing species, to get all H-bearing reactions

In [113]:
bigdf_Hrxns = DataFrame(Rxn=[], Value=[])

# Retrieve 100% of reactions for all species
for Hsp in H_bearing_species
    # Get the basic df
    println("For $(Hsp): ")
    col_rate_df = get_column_rates_NEW(Hsp, ncur; which="all", Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)
    
    thisdf = get_p_pct_of_column(col_rate_df, 1; Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)
    bigdf_Hrxns = vcat(bigdf_Hrxns, thisdf)
    println()
end

For ArHpl: 
100% of reactions is 16/16
The column rate of top 100.0% of reactions is 10012.687993655469
Which equates to 100.0% of the total column rate (10012.687993655469).

16×2 DataFrame
 Row │ Rxn                          Value          
     │ String                       Float64        
─────┼─────────────────────────────────────────────
   1 │ H2pl + Ar --> ArHpl + H      4987.98
   2 │ ArHpl + CO2 --> HCO2pl + Ar  3882.06
   3 │ ArHpl + O --> OHpl + Ar       513.096
   4 │ ArHpl + N2 --> N2Hpl + Ar     314.979
   5 │ ArHpl + CO --> HCOpl + Ar     261.359
   6 │ ArHpl + H2 --> H3pl + Ar       20.952
   7 │ ArHpl + O2 --> HO2pl + Ar      13.225
   8 │ H3pl + Ar --> ArHpl + H2        8.72566
   9 │ HDpl + Ar --> ArHpl + D         8.03077
  10 │ Arpl + H2 --> ArHpl + H         1.58559
  11 │ ArHpl + C --> CHpl + Ar         0.327474
  12 │ ArHpl + E --> Ar + H            0.305297
  13 │ ArHpl + HD --> H2Dpl + Ar       0.0382802
  14 │ ArDpl + H2 --> ArHpl + HD       0.0214036
  15 

# Figure out which reactions need deuterated analogue by collecting reactions sufficient to comprise 99% of this total column rate

## Take H-bearing rxn dataframe, filter out duplicates, and sort it by column value

In [114]:
bigdf_Hrxns_sorted_uniques = unique(sort(bigdf_Hrxns, [:Value], rev=true))

,Rxn,Value
,Any,Any
1,H + O2 --> HO2,1.4822e12
2,CO + OH --> CO2 + H,1.344e12
3,HO2 + O --> O2 + OH,1.34282e12
4,OH + O --> O2 + H,2.05675e11
5,O3 + H --> O2 + OH,8.61523e10
6,HO2 + H --> OH + OH,4.35459e10
7,H2O --> H + OH,3.24901e10
8,HO2 + OH --> H2O + O2,3.06452e10
9,HO2 + HO2 --> H2O2 + O2,2.27018e10


## Filter out any reactions that already are deuterated

In [115]:
# This is what we will filter out:
# blah = filter(row->any(x->occursin(x, row."Rxn"), [r"^D ", " D", "DO2", "HD", "DCO", "Dpl", "H2D", "OD"]), bigdf_sorted_uniques)
# for row in eachrow(blah)
#     println(row."Rxn")
# end
# println(nrow(blah))

deuteration_candidates = filter(row->!any(x->occursin(x, row."Rxn"), [r"^D ", " D", "DO2", "HD", "DCO", "Dpl", "H2D", "OD"]), bigdf_Hrxns_sorted_uniques)

# for row in eachrow(needs_d_analogue)
#     println(row."Rxn")
# end

,Rxn,Value
,Any,Any
1,H + O2 --> HO2,1.4822e12
2,CO + OH --> CO2 + H,1.344e12
3,HO2 + O --> O2 + OH,1.34282e12
4,OH + O --> O2 + H,2.05675e11
5,O3 + H --> O2 + OH,8.61523e10
6,HO2 + H --> OH + OH,4.35459e10
7,H2O --> H + OH,3.24901e10
8,HO2 + OH --> H2O + O2,3.06452e10
9,HO2 + HO2 --> H2O2 + O2,2.27018e10


## Retrieve only the reactions which comprise top 99.99% of the column rate

In [116]:
top_rxns = rxns_comprising_p_pct_colrate(deuteration_candidates, 0.9999; Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)

282 is 100.0%
281 is 100.0%
280 is 100.0%
279 is 100.0%
278 is 100.0%
277 is 100.0%
276 is 100.0%
275 is 100.0%
274 is 100.0%
273 is 100.0%
272 is 100.0%
271 is 100.0%
270 is 100.0%
269 is 100.0%
268 is 100.0%
267 is 100.0%
266 is 100.0%
265 is 100.0%
264 is 100.0%
263 is 100.0%
262 is 100.0%
261 is 100.0%
260 is 100.0%
259 is 100.0%
258 is 100.0%
257 is 100.0%
256 is 100.0%
255 is 100.0%
254 is 99.99999999999997%
253 is 99.99999999999996%
252 is 99.99999999999993%
251 is 99.99999999999991%
250 is 99.9999999999999%
249 is 99.99999999999986%
248 is 99.99999999999982%
247 is 99.99999999999977%
246 is 99.99999999999972%
245 is 99.99999999999967%
244 is 99.9999999999996%
243 is 99.99999999999952%
242 is 99.99999999999939%
241 is 99.9999999999992%
240 is 99.999999999999%
239 is 99.99999999999882%
238 is 99.99999999999854%
237 is 99.9999999999979%
236 is 99.99999999999727%
235 is 99.99999999999662%
234 is 99.99999999999592%
233 is 99.99999999999487%
232 is 99.99999999999383%
231 is 99.999999

,Rxn,Value
,Any,Any
1,H + O2 --> HO2,1.4822e12
2,CO + OH --> CO2 + H,1.344e12
3,HO2 + O --> O2 + OH,1.34282e12
4,OH + O --> O2 + H,2.05675e11
5,O3 + H --> O2 + OH,8.61523e10
6,HO2 + H --> OH + OH,4.35459e10
7,H2O --> H + OH,3.24901e10
8,HO2 + OH --> H2O + O2,3.06452e10
9,HO2 + HO2 --> H2O2 + O2,2.27018e10


# Now get the deuterated analogues

In [86]:
D_bearing_species = [s for s in setdiff(union(vardict["neutral_species"], vardict["ion_species"]), [:O1D, :Nup2D]) if occursin('D', string(s))];

In [108]:
function deuterated_combinations(s; D_to_H=false)
    #=
    Given a reactant or product string (i.e. one set of prodcuts),
    this figures out all the possible ways to singly deuterate it.
    No duplicates.
    =#
    
    prods_split = split(s, " + ")

    N = length(prods_split)
    n = collect(1:N)

    newopts = Array{String}(undef, N, N)

    # Fill in all possible options by changing to deuterated analogue of products, one by one
    for i in 1:N
        newopts[i, :] = [prods_split[j] for j in n] # fills in the array with the products
        newopts[i, i] = H_to_D(newopts[i, i]; reverse=D_to_H) 
    end
    
    # Blank out any rows for which the product sets are functionally the same
    for i in 1:N
        for j in i+1:N
            if Set(newopts[i, :]) == Set(newopts[j, :])
                newopts[j, :] .= ""
            end
        end
    end
    
    # Delete the blank rows
    deletetheserows = []
    if D_to_H == true 
        for i in 1:N      
            # get rid of any rows that have even one D-bearing species or a blank reactant/product
            # (takes care of rows that have a rogue O1D, and blank rows)
            if any(x->in(Symbol(x), D_bearing_species), newopts[i, :]) | any(x->x=="", newopts[i, :])
                append!(deletetheserows, i)
            end
        end
    else
        for i in 1:N      
            # get rid of any rows that don't have a D-bearing species 
            # (takes care of rows that have a rogue O1D, and blank rows)
            if all(x->!in(Symbol(x), D_bearing_species), newopts[i, :])
                append!(deletetheserows, i)
            end
        end
    end

    newopts = newopts[setdiff(1:end, deletetheserows), :]
    new_prod_str = []
    for i in 1:size(newopts)[1] # length has changed
        push!(new_prod_str, join(newopts[i, :], " + "))
    end

    return new_prod_str
end

function H_to_D(s; count=100, reverse=false)

    if reverse==true
        return replace(s, "O1D"=>"O1D", "H2D"=>"H3", "HD"=>"H2", "D"=>"H"; count=count) # This makes sure not to fuck up O1D
    else
        return replace(s, "H3"=>"H2D", "H2"=>"HD", "H"=>"D"; count=count)
    end
end



H_to_D (generic function with 1 method)

## Testing...

In [20]:
deuterated_combinations("OH + H + H")

2-element Vector{Any}:
 "OD + H + H"
 "OH + D + H"

In [21]:
deuterated_combinations("OH + OH")

1-element Vector{Any}:
 "OD + OH"

In [22]:
deuterated_combinations("H + H + H")

1-element Vector{Any}:
 "D + H + H"

## Code that allows us to automatically create the deuterated analogue reactions

### Good code

In [98]:
function translate_D_and_H(rxndf; D_to_H=false)
    #=
    Take a dataframe containing reactions and their column values and replace H with D to form deuterated analogues.
    =#
    
    look_for_these = []
    
    if D_to_H == true
        replaced_atom = "D"
    else
        replaced_atom = "H"
    end
    
    for r in eachrow(rxndf)
        reactants, products = decompose_chemistry_string(r[1]; returntype="strings")
        
        #Count H in the reactants and products
        H_or_D_in_reactants = count("replaced_atom", reactants)
        H_or_D_in_products = count("replaced_atom", products)
        
        if (H_or_D_in_reactants == 1) & (H_or_D_in_products == 1)
            new_reacts = H_to_D(reactants; reverse=D_to_H)
            new_prods = H_to_D(products; reverse=D_to_H)
            println("$(r[1]) -----> $(new_reacts) --> $(new_prods)")
            
            new_reacts_symb = [Symbol(r) for r in split(new_reacts, " + ")]
            new_prods_symb = [Symbol(r) for r in split(new_prods, " + ")]
            
            push!(look_for_these, [new_reacts_symb, new_prods_symb])
            println()
        else
            new_reacts = deuterated_combinations(reactants; D_to_H=D_to_H)
            new_prods = deuterated_combinations(products; D_to_H=D_to_H)
            
            println("Deuteration options for $(reactants) --> $(products)")
            for R in new_reacts
                for P in new_prods
                    new_reacts_symb = [Symbol(r) for r in split(R, " + ")]
                    new_prods_symb = [Symbol(p) for p in split(P, " + ")]
                    
                    # push!(look_for_these, "$(r) --> $(p)")
                    push!(look_for_these, [new_reacts_symb, new_prods_symb])
                    println("$(R) --> $(P)")
                end
            end

            println()
        
        end
    end
    return look_for_these
end



translate_D_and_H (generic function with 1 method)

In [99]:
look_for_these = translate_D_and_H(top_rxns."Rxn")

Deuteration options for H + O2 --> HO2
D + O2 --> DO2

Deuteration options for CO + OH --> CO2 + H
CO + OD --> CO2 + D

Deuteration options for HO2 + O --> O2 + OH
DO2 + O --> O2 + OD

Deuteration options for OH + O --> O2 + H
OD + O --> O2 + D

Deuteration options for O3 + H --> O2 + OH
O3 + D --> O2 + OD

Deuteration options for HO2 + H --> OH + OH
DO2 + H --> OD + OH
HO2 + D --> OD + OH

Deuteration options for H2O --> H + OH
HDO --> D + OH
HDO --> H + OD

Deuteration options for HO2 + OH --> H2O + O2
DO2 + OH --> HDO + O2
HO2 + OD --> HDO + O2

Deuteration options for HO2 + HO2 --> H2O2 + O2
DO2 + HO2 --> HDO2 + O2

Deuteration options for H2O2 --> OH + OH
HDO2 --> OD + OH

Deuteration options for HO2 --> OH + O
DO2 --> OD + O

Deuteration options for CO + OH --> HOCO
CO + OD --> DOCO

Deuteration options for HOCO + O2 --> CO2 + HO2
DOCO + O2 --> CO2 + DO2

Deuteration options for HO2 + H --> O2 + H2
DO2 + H --> O2 + HD
HO2 + D --> O2 + HD

Deuteration options for O1D + H2O --> OH 

36-element Vector{Any}:
 [[:D, :O2], [:DO2]]
 [[:CO, :OD], [:CO2, :D]]
 [[:DO2, :O], [:O2, :OD]]
 [[:OD, :O], [:O2, :D]]
 [[:O3, :D], [:O2, :OD]]
 [[:DO2, :H], [:OD, :OH]]
 [[:HO2, :D], [:OD, :OH]]
 [[:HDO], [:D, :OH]]
 [[:HDO], [:H, :OD]]
 [[:DO2, :OH], [:HDO, :O2]]
 [[:HO2, :OD], [:HDO, :O2]]
 [[:DO2, :HO2], [:HDO2, :O2]]
 [[:HDO2], [:OD, :OH]]
 ⋮
 [[:OD, :H2O2], [:H2O, :DO2]]
 [[:OH, :HDO2], [:HDO, :HO2]]
 [[:OH, :HDO2], [:H2O, :DO2]]
 [[:HDO2], [:DO2, :H]]
 [[:HDO2], [:HO2, :D]]
 [[:DO2, :H], [:HDO, :O1D]]
 [[:HO2, :D], [:HDO, :O1D]]
 [[:O3, :DO2], [:O2, :O2, :OD]]
 [[:DO2, :HO2, :M], [:HDO2, :O2, :M]]
 [[:DCOpl, :E], [:CO, :D]]
 [[:O1D, :HD], [:OD, :H]]
 [[:O1D, :HD], [:OH, :D]]

# Search for the deuterated analogues in the network

In [121]:
function reactions_present(rxns_to_find; return_found=false, reaction_network)
    #=
    rxns_to_find: list of reactions like [[reactants], [products]] (symbols) that
    are deuterated analogues that we should have in reaction_network. reports how many
    are not there and what they are.
    WARNING: It is finding an O1D reaction that it shouldn't be check each to be sure
    =#
    
    missing_count = 0
    not_found = []
    
    found = []

    flag = false
    for rxn in rxns_to_find
        for rxn_in_net in reaction_network
            if (Set(rxn[1])==Set(rxn_in_net[1])) & (Set(rxn[2])==Set(rxn_in_net[2]))
                # println("Found $(format_chemistry_string(rxn[1], rxn[2])) match: $(format_chemistry_string(rxn_in_net[1], rxn_in_net[2]))")
                flag = true
                push!(found, rxn_in_net)
            end
        end
        if flag == false
            push!(not_found, rxn)
            # println("\n !!!!!!!!!!!!!!!!! \n $(rxn) not found in the network\n !!!!!!!!!!!!!!!!! \n ")
            missing_count += 1
        end
        flag = false
    end
    println("Missing $(missing_count) reactions")
    
    if return_found
        if missing_count > 0
            println("These are the ones I couldn't find, suspiciously")
            println(not_found)
            println()
        end
        return found
    else
        return not_found
    end
end

reactions_present (generic function with 1 method)

In [90]:
not_found = reactions_present(look_for_these; reaction_network)

Missing 2 reactions


2-element Vector{Any}:
 [[:OD, :H2], [:H2O, :D]]
 [[:OD, :H2O2], [:H2O, :DO2]]

Wait a second--that first reaction isn't possible. I think the code is coming up with garbage. Need to review the auto-creation of the analogues...

# Figure out exactly how much of the total H column the H analogues of our actual D network cover.

In [109]:
bigdf_Drxns = DataFrame(Rxn=[], Value=[])

# Retrieve 100% of reactions for all species
for Dsp in D_bearing_species
    # Get the basic df
    println("For $(Dsp): ")
    col_rate_df = get_column_rates_NEW(Dsp, ncur; which="all", Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)
    
    thisdf = get_p_pct_of_column(col_rate_df, 1; Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)
    bigdf_Drxns = vcat(bigdf_Drxns, thisdf)
    println()
end

For D: 
100% of reactions is 84/84
The column rate of top 100.0% of reactions is 9.845765378507174e8
Which equates to 100.0% of the total column rate (9.845765378507174e8).

84×2 DataFrame
 Row │ Rxn                         Value            
     │ String                      Float64          
─────┼──────────────────────────────────────────────
   1 │ D + O2 --> DO2                   4.02785e8
   2 │ CO + OD --> CO2 + D              3.87903e8
   3 │ O + OD --> O2 + D                9.10775e7
   4 │ D + O3 --> OD + O2               3.2716e7
   5 │ D + HO2 --> DO2 + H              2.61155e7
   6 │ OH + D --> OD + H                2.00481e7
   7 │ D + HO2 --> OH + OD              1.33503e7
   8 │ HDO --> D + OH                   8.49736e6
   9 │ D + HO2 --> HD + O2              6.397e5
  10 │ OD + H --> OH + D                3.14005e5
  11 │ OH + HD --> H2O + D         300805.0
  12 │ D + HO2 --> HDO + O1D            2.96673e5
  13 │ H + DO2 --> HO2 + D         228670.0
  14 │ HDO2 --> H

In [110]:
bigdf_Drxns_sorted_uniques = unique(sort(bigdf_Drxns, [:Value], rev=true))

back_to_H_version = unique(translate_D_and_H(bigdf_Drxns_sorted_uniques."Rxn"; D_to_H=true))

Deuteration options for D + O2 --> DO2
H + O2 --> HO2

Deuteration options for O + DO2 --> OD + O2
O + HO2 --> OH + O2

Deuteration options for CO + OD --> CO2 + D
CO + OH --> CO2 + H

Deuteration options for O + OD --> O2 + D
O + OH --> O2 + H

Deuteration options for D + O3 --> OD + O2
H + O3 --> OH + O2

Deuteration options for D + HO2 --> DO2 + H
H + HO2 --> HO2 + H

Deuteration options for OH + D --> OD + H
OH + H --> OH + H

Deuteration options for H + DO2 --> OH + OD
H + HO2 --> OH + OH

Deuteration options for D + HO2 --> OH + OD
H + HO2 --> OH + OH

Deuteration options for OH + DO2 --> HDO + O2
OH + HO2 --> H2O + O2

Deuteration options for OD + HO2 --> HDO + O2
OH + HO2 --> H2O + O2

Deuteration options for HDO --> D + OH
H2O --> H + OH

Deuteration options for HDO --> H + OD
H2O --> H + OH

Deuteration options for DO2 + HO2 --> HDO2 + O2
HO2 + HO2 --> H2O2 + O2

Deuteration options for HDO2 --> OH + OD
H2O2 --> OH + OH

Deuteration options for DO2 --> OD + O
HO2 --> OH + O



142-element Vector{Any}:
 [[:H, :O2], [:HO2]]
 [[:O, :HO2], [:OH, :O2]]
 [[:CO, :OH], [:CO2, :H]]
 [[:O, :OH], [:O2, :H]]
 [[:H, :O3], [:OH, :O2]]
 [[:H, :HO2], [:HO2, :H]]
 [[:OH, :H], [:OH, :H]]
 [[:H, :HO2], [:OH, :OH]]
 [[:OH, :HO2], [:H2O, :O2]]
 [[:H2O], [:H, :OH]]
 [[:HO2, :HO2], [:H2O2, :O2]]
 [[:H2O2], [:OH, :OH]]
 [[:HO2], [:OH, :O]]
 ⋮
 [[:Cpl, :H2O], [:HCOpl, :H]]
 [[:Cpl, :H2O], [:H2Opl, :C]]
 [[:H3pl, :H2], [:H3pl, :H2]]
 [[:H2O], [:Opl, :H2]]
 [[:N2pl, :H2O], [:N2Hpl, :OH]]
 [[:Hpl, :NO], [:NOpl, :H]]
 [[:Arpl, :H2], [:ArHpl, :H]]
 [[:Hpl, :H2O], [:H2Opl, :H]]
 [[:Arpl, :H2], [:H2pl, :Ar]]
 [[:Cpl, :H2], [:CHpl, :H]]
 [[:H2O], [:H, :H, :O]]
 [[:H2O2], [:H2O, :O1D]]

In [122]:
# Retrieve the list of reactions from the network with the rates

reconstructed_H_network = reactions_present(back_to_H_version; return_found=true, reaction_network)

Missing 10 reactions
These are the ones I couldn't find, suspiciously
Any[[[:H, :HO2], [:HO2, :H]], [[:OH, :H], [:OH, :H]], [[:H, :H2], [:H2, :H]], [[:Hpl, :CO2], [:CO2pl, :H]], [[:HCOpl, :H], [:HCOpl, :H]], [[:N2pl, :H], [:Hpl, :N2]], [[:Hpl, :H2], [:Hpl, :H2]], [[:N2Hpl, :H], [:N2Hpl, :H]], [[:ArHpl, :H2], [:ArHpl, :H2]], [[:H3pl, :H2], [:H3pl, :H2]]]



134-element Vector{Any}:
 Any[[:H, :O2], [:HO2], :((((1.4613e-28 .* Tn .^ -1.3) .* M) ./ (1 .+ ((1.4613e-28 .* Tn .^ -1.3) .* M) ./ (2.39683e-11 .* Tn .^ 0.2))) .* 0.6 .^ ((1 .+ log10.(((1.4613e-28 .* Tn .^ -1.3) .* M) ./ (2.39683e-11 .* Tn .^ 0.2)) .^ 2) .^ -1.0))]
 Any[[:HO2, :O], [:O2, :OH], :(3.0e-11 .* exp.(200 ./ Tn))]
 Any[[:CO, :OH], [:CO2, :H], :(((4.89574181180489e-15 .* Tn .^ 0.6) ./ (1 .+ (4.89574181180489e-15 .* Tn .^ 0.6) ./ ((1.62846954489706e-6 .* Tn .^ 6.1) ./ M))) .* 0.6 .^ ((1 .+ log10.((4.89574181180489e-15 .* Tn .^ 0.6) ./ ((1.62846954489706e-6 .* Tn .^ 6.1) .* M)) .^ 2) .^ -1.0))]
 Any[[:OH, :O], [:O2, :H], :(1.8e-11 .* exp.(180 ./ Tn))]
 Any[[:O3, :H], [:O2, :OH], :(1.4e-10 .* exp.(-470 ./ Tn))]
 Any[[:HO2, :H], [:OH, :OH], 7.2e-11]
 Any[[:HO2, :OH], [:H2O, :O2], :(4.8e-11 .* exp.(250 ./ Tn))]
 Any[[:H2O], [:H, :OH], :JH2OtoHpOH]
 Any[[:HO2, :HO2], [:H2O2, :O2], :(3.0e-13 .* exp.(460 ./ Tn))]
 Any[[:H2O2], [:OH, :OH], :JH2O2toOHpOH]
 Any[[:HO2], [:OH, :O], :JHO2t

I've decided I don't care about those 10 reactions. I'll just calculate the percent of the total rate for the found ones and call it a day

## Send the reconstructed H diagram back into the column rates calculator...

In [125]:
species_with_ONLY_H = setdiff(H_bearing_species, D_bearing_species)

23-element Vector{Symbol}:
 :ArHpl
 :CHpl
 :H
 :H2
 :H2O
 :H2O2
 :H2Opl
 :H2pl
 :H3Opl
 :H3pl
 :HCO
 :HCO2pl
 :HCOpl
 :HNOpl
 :HO2
 :HO2pl
 :HOCO
 :HOCpl
 :Hpl
 :N2Hpl
 :NHpl
 :OH
 :OHpl

In [140]:
bigdf_H_reconstructed = DataFrame(Rxn=[], Value=[])

# Retrieve 100% of reactions for all species
for Hsp in species_with_ONLY_H
    # Get the basic df
    println("For $(Hsp): ")
    col_rate_df = get_column_rates_NEW(Hsp, ncur; which="all", Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reconstructed_H_network, # !!!!!!!!!!! the change
                                   num_layers, dz)
    
    thisdf = get_p_pct_of_column(col_rate_df, 1; Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reconstructed_H_network, # !!!!!!!!!!! the change
                                   num_layers, dz)
    bigdf_H_reconstructed = vcat(bigdf_H_reconstructed, thisdf)
    println()
end

bigdf_H_reconstructed = unique(sort(bigdf_H_reconstructed, [:Value], rev=true))

For ArHpl: 
100% of reactions is 6/6
The column rate of top 100.0% of reactions is 9468.91689737013
Which equates to 100.0% of the total column rate (9468.91689737013).

6×2 DataFrame
 Row │ Rxn                          Value      
     │ String                       Float64    
─────┼─────────────────────────────────────────
   1 │ H2pl + Ar --> ArHpl + H      4987.98
   2 │ ArHpl + CO2 --> HCO2pl + Ar  3882.06
   3 │ ArHpl + N2 --> N2Hpl + Ar     314.979
   4 │ ArHpl + CO --> HCOpl + Ar     261.359
   5 │ ArHpl + H2 --> H3pl + Ar       20.952
   6 │ Arpl + H2 --> ArHpl + H         1.58559

For CHpl: 
100% of reactions is 1/1
The column rate of top 100.0% of reactions is 4.352715331858977
Which equates to 100.0% of the total column rate (4.352715331858977).

1×2 DataFrame
 Row │ Rxn                    Value   
     │ String                 Float64 
─────┼────────────────────────────────
   1 │ Cpl + H2 --> CHpl + H  4.35272

For H: 
100% of reactions is 68/68
The column rate of top 10

,Rxn,Value
,Any,Any
1,H + O2 --> HO2,1.4822e12
2,CO + OH --> CO2 + H,1.344e12
3,HO2 + O --> O2 + OH,1.34282e12
4,OH + O --> O2 + H,2.05675e11
5,O3 + H --> O2 + OH,8.61523e10
6,HO2 + H --> OH + OH,4.35459e10
7,H2O --> H + OH,3.24901e10
8,HO2 + OH --> H2O + O2,3.06452e10
9,HO2 + HO2 --> H2O2 + O2,2.27018e10


In [141]:
PERCENT_OF_H_COLUMN_EQUIVALENT = sum(bigdf_H_reconstructed."Value") / sum(deuteration_candidates."Value")

0.9999996592038692

In [129]:
deuteration_candidates

,Rxn,Value
,Any,Any
1,H + O2 --> HO2,1.4822e12
2,CO + OH --> CO2 + H,1.344e12
3,HO2 + O --> O2 + OH,1.34282e12
4,OH + O --> O2 + H,2.05675e11
5,O3 + H --> O2 + OH,8.61523e10
6,HO2 + H --> OH + OH,4.35459e10
7,H2O --> H + OH,3.24901e10
8,HO2 + OH --> H2O + O2,3.06452e10
9,HO2 + HO2 --> H2O2 + O2,2.27018e10


# Dominant reactions for H

In [208]:
print_dominant_reactions(ncur, :H, 0.33; use_ion=false, Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)

Column rates will be collected for reaction type = krates and species role = both
The sum of all 145 reactions is: 3.166825049012297e12
33.0% of reactions is 47/145
The column rate of top 33.0% of reactions is 3.166824869036046e12
Which equates to 99.99999431682367% of the total column rate.

47×2 DataFrame
 Row │ Rxn                         Value           
     │ String                      Float64         
─────┼─────────────────────────────────────────────
   1 │ H + O2 --> HO2                   1.4822e12
   2 │ CO + OH --> CO2 + H              1.344e12
   3 │ OH + O --> O2 + H                2.05675e11
   4 │ O3 + H --> O2 + OH               8.61523e10
   5 │ HO2 + H --> OH + OH              4.35459e10
   6 │ HO2 + H --> O2 + H2              2.08658e9
   7 │ OH + H2 --> H2O + H              1.61533e9
   8 │ HO2 + H --> H2O + O1D            9.67687e8
   9 │ HCOpl + E --> CO + H             1.42594e8
  10 │ O1D + H2 --> OH + H              1.32058e8
  11 │ CO2pl + H2 --> HCO2pl + H 

# Dominant reactions for H2O

In [209]:
print_dominant_reactions(ncur, :H2O, 0.33; use_ion=false, Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)

Column rates will be collected for reaction type = krates and species role = both
The sum of all 57 reactions is: 3.599391832775264e10
33.0% of reactions is 18/57
The column rate of top 33.0% of reactions is 3.59939176113226e10
Which equates to 99.99999800958031% of the total column rate.

18×2 DataFrame
 Row │ Rxn                           Value           
     │ String                        Float64         
─────┼───────────────────────────────────────────────
   1 │ HO2 + OH --> H2O + O2              3.06452e10
   2 │ O1D + H2O --> OH + OH              1.62693e9
   3 │ OH + H2 --> H2O + H                1.61533e9
   4 │ OH + H2O2 --> H2O + HO2            1.09603e9
   5 │ HO2 + H --> H2O + O1D              9.67687e8
   6 │ OH + OH --> H2O + O                3.56231e7
   7 │ OH + H + CO2 --> H2O + CO2         3.69939e6
   8 │ H + H2O2 --> H2O + OH              1.42154e6
   9 │ H2O2 + H --> H2O + OH              1.42154e6
  10 │ OH + HD --> H2O + D           300805.0
  11 │ OH + HDO2 

# Dominant reactions for OH

In [210]:
print_dominant_reactions(ncur, :OH, 0.33; use_ion=false, Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)

Column rates will be collected for reaction type = krates and species role = both
The sum of all 102 reactions is: 3.063854424653401e12
33.0% of reactions is 33/102
The column rate of top 33.0% of reactions is 3.063853168714076e12
Which equates to 99.99995900786557% of the total column rate.

33×2 DataFrame
 Row │ Rxn                         Value           
     │ String                      Float64         
─────┼─────────────────────────────────────────────
   1 │ CO + OH --> CO2 + H              1.344e12
   2 │ HO2 + O --> O2 + OH              1.34282e12
   3 │ OH + O --> O2 + H                2.05675e11
   4 │ O3 + H --> O2 + OH               8.61523e10
   5 │ HO2 + H --> OH + OH              4.35459e10
   6 │ HO2 + OH --> H2O + O2            3.06452e10
   7 │ CO + OH --> HOCO                 5.95397e9
   8 │ O1D + H2O --> OH + OH            1.62693e9
   9 │ OH + H2 --> H2O + H              1.61533e9
  10 │ OH + H2O2 --> H2O + HO2          1.09603e9
  11 │ O3 + HO2 --> O2 + O2 + O

# Dominant reactions for HCO

In [211]:
print_dominant_reactions(ncur, :HCO, 0.33; use_ion=true, Tn=vardict["Tn_arr"][2:end-1], Ti=vardict["Ti_arr"][2:end-1], Te=vardict["Te_arr"][2:end-1], 
                                   all_species=vardict["all_species"], ion_species=vardict["ion_species"], reaction_network=reaction_network,
                                   num_layers, dz)

Column rates will be collected for reaction type = krates and species role = both
The sum of all 31 reactions is: 5.242318647211368e7
33.0% of reactions is 10/31
The column rate of top 33.0% of reactions is 5.242318481440615e7
Which equates to 99.99999683783524% of the total column rate.

10×2 DataFrame
 Row │ Rxn                        Value         
     │ String                     Float64       
─────┼──────────────────────────────────────────
   1 │ CO + H --> HCO                 2.61704e7
   2 │ HCO + O2 --> HO2 + CO          2.22961e7
   3 │ HCO + O2 --> CO2 + OH          3.25867e6
   4 │ HCO + O --> CO + OH            3.41079e5
   5 │ HCO + O --> CO2 + H            3.41079e5
   6 │ HCO + H --> CO + H2        14080.3
   7 │ NHpl + CO2 --> NOpl + HCO   1635.31
   8 │ HCO + OH --> H2O + CO        132.454
   9 │ HCO + D --> CO + HD            5.67946
  10 │ O2pl + HCO --> HCOpl + O2      1.60158
